In [4]:
!pip install --upgrade keras
!pip install --upgrade keras tensorflow
import os
import math
import numpy as np
import librosa
import json
import pandas as pd
from termcolor import colored
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
!pip install tensorboard
!pip install visualkeras
import visualkeras
from keras.utils import plot_model
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.svm import SVC
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from IPython.display import Audio
from scipy.io import wavfile
import scipy
import soundfile as sf
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
import random

import csv
from keras.callbacks import Callback
from keras.callbacks import CSVLogger
from sklearn.metrics import precision_recall_fscore_support

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Unknown_Spectrograms/Train/'
test_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Unknown_Spectrograms/Test/'

num_classes = 7
height = 128
n_fft = 2048
hop_length = 512
sr = 16000
T = 1

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

width = 1 + int((audio_length_samples - n_fft) / hop_length)

channels = 3

train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator()

batch_size = 32

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical')

model = tf.keras.Sequential()

# Convolutional layers
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Flatten layer to feed into fully connected layers
model.add(tf.keras.layers.Flatten())

# Dense layers
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'precision', 'recall', 'f1_score'])

epochs = 200
metrics_data = []

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    train_generator.reset()

    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=1,
        verbose=1)

    accuracy = history.history['accuracy'][0]
    precision = history.history['precision'][0]
    recall = history.history['recall'][0]

    f1_score_tensor = history.history['f1_score'][0]
    f1_score_value = np.mean(f1_score_tensor)

    metrics_data.append([accuracy, precision, recall, f1_score_value])

csv_filename = 'training_metrics.csv'

with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Accuracy', 'Precision', 'Recall', 'F1 Score'])
    writer.writerows(metrics_data)

print(f"Training metrics saved to {csv_filename}")

loss, accuracy, precision, recall, f1_score = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Test Precision:", precision)
print("Test Recall:", recall)
print("Test F1 Score:", f1_score)

model.save('el-amadi_cnn_standardized_unknown_trial2.keras')

Found 3088 images belonging to 7 classes.
Found 768 images belonging to 7 classes.
Epoch 1/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 76s 704ms/step - accuracy: 0.3396 - f1_score: 0.1637 - loss: 17.1379 - precision: 0.4057 - recall: 0.2306
Epoch 2/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 74s 711ms/step - accuracy: 0.5357 - f1_score: 0.2921 - loss: 1.2571 - precision: 0.7708 - recall: 0.3748
Epoch 3/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 82s 789ms/step - accuracy: 0.6650 - f1_score: 0.5230 - loss: 0.9639 - precision: 0.8276 - recall: 0.5204
Epoch 4/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 75s 725ms/step - accuracy: 0.7294 - f1_score: 0.6148 - loss: 0.7840 - precision: 0.8421 - recall: 0.6208
Epoch 5/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 74s 712ms/step - accuracy: 0.7606 - f1_score: 0.6591 - loss: 0.6958 - precision: 0.8587 - recall: 0.6618
Epoch 6/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 73s 703ms/step - accuracy: 0.7856 - f1_score: 0.6963 - loss: 0.5927 - precision: 0.8562 - recall: 0.7196
Epoch 7/200
97/97 ━━━━━━━━━━━━━━━━━━━━ 74s 714ms/ste

In [8]:
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

num_classes = 7
height = 128
n_fft = 2048
hop_length = 512
sr = 16000
T = 1

hop_size_sec = hop_length / sr
audio_length_samples = T * sr

width = 1 + int((audio_length_samples - n_fft) / hop_length)

channels = 3

batch_size = 32

model = tf.keras.models.load_model('/content/el-amadi_cnn_standardized_unknown_trial2.keras')

test_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Unknown_Spectrograms/Test/'

test_datagen = ImageDataGenerator()

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(height, width),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

loss, accuracy, precision, recall, f1_score = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Test Precision:", precision)
print("Test Recall:", recall)
print("Test F1 Score:", f1_score)

predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

# Get true class labels
true_classes = test_generator.classes

# Compute confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("\nConfusion Matrix:")
print(conf_matrix)

# Get class labels
class_labels = list(test_generator.class_indices.keys())

# Calculate accuracy for each class
class_accuracy = {}
for i, class_label in enumerate(class_labels):
    correct_indices = np.where(true_classes == i)[0]
    correct_predictions = np.sum(predicted_classes[correct_indices] == i)
    total_samples = len(correct_indices)
    class_accuracy[class_label] = correct_predictions / total_samples if total_samples > 0 else 0.0

# Print accuracy for each class
print("\nAccuracy for each class:")
for class_label in class_labels:
    print(f"{class_label}: {class_accuracy[class_label]:.4f}")

# Print classification report (includes precision, recall, F1-score)
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=class_labels))

Found 768 images belonging to 7 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


24/24 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.9653 - f1_score: 0.6501 - loss: 0.1296 - precision: 0.9676 - recall: 0.9636
Test Loss: 0.17688150703907013
Test Accuracy: 0.9505208134651184
Test Precision: 0.9529411792755127
Test Recall: 0.94921875
Test F1 Score: tf.Tensor(
[0.97169805 1.         0.88372093 1.         0.903846   0.9526541
 0.98076916], shape=(7,), dtype=float32)
24/24 ━━━━━━━━━━━━━━━━━━━━ 14s 595ms/step

Confusion Matrix:
[[103   0   1   0   0   0   0]
 [  0  52   0   0   0   0   0]
 [  2   0  95   0   0   7   0]
 [  0   0   0  50   0   0   0]
 [  0   0   0   0  47   1   2]
 [  3   0  15   0   7 332   0]
 [  0   0   0   0   0   0  51]]

Accuracy for each class:
Bebop: 0.9904
Crazyflie: 1.0000
Mambo: 0.9135
Matrice100: 1.0000
Mavic: 0.9400
NonDrone: 0.9300
Phantom: 1.0000

Classification Report:
              precision    recall  f1-score   support

       Bebop       0.95      0.99      0.97       104
   Crazyflie       1.00      1.00      1.00        52
       Mam

In [20]:
import os
import numpy as np
from scipy.stats import entropy
from tensorflow.keras.preprocessing import image

model = tf.keras.models.load_model('/content/el-amadi_cnn_standardized_unknown_trial2.keras')
classes = ['Bebop', 'Crazyflie', 'Mambo', 'Matrice100', 'Mavic', 'NonDrone', 'Phantom']

known_counter = 0
unknown_counter = 0

def load_spectrogram(file_path):
    # Load and preprocess the image
    num_classes = 7
    height = 128
    n_fft = 2048
    hop_length = 512
    sr = 16000
    T = 1

    hop_size_sec = hop_length / sr
    audio_length_samples = T * sr

    width = 1 + int((audio_length_samples - n_fft) / hop_length)

    channels = 3

    img = image.load_img(file_path, target_size=(height, width))  # Assuming input size expected by your model
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    #img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
    return img_array

# Function to classify a single spectrogram
def classify_spectrogram(spectrogram, entropy_threshold):
    global known_counter
    global unknown_counter

    probabilities = model.predict(spectrogram)[0]  # Get class probabilities from the model
    ent = entropy(probabilities, base=2)  # Calculate entropy of the probability distribution

    if ent > entropy_threshold:
        unknown_counter+=1
        return "UnknownDrone"
    else:
        known_counter+=1
        max_index = np.argmax(probabilities)  # Get the index of the maximum probability
        return classes[max_index]  # Return the corresponding drone class

# Directory containing spectrogram files
unknown_spectrogram_dir = '/content/drive/MyDrive/UnknownDrone/'
spectrogram_dir = '/content/drive/MyDrive/Colab Notebooks/TAMUCC REU 2024/DS1/Al-Emadi_Standardized_Unknown_Spectrograms/Test/Phantom/'

# Entropy threshold
entropy_threshold =  0.00002 # Adjust this threshold based on your model and validation results

# List all files in the directory
spectrogram_files = os.listdir(spectrogram_dir)

# Process each spectrogram file
for filename in spectrogram_files:
    if filename.endswith('.png'):  # Assuming spectrograms are saved as PNG files
        # Load the spectrogram (you need to implement this part based on your data loading method)
        spectrogram = load_spectrogram(os.path.join(spectrogram_dir, filename))

        # Classify the spectrogram
        predicted_class = classify_spectrogram(spectrogram, entropy_threshold)

        # Print the filename and predicted class
        print(f"{filename}: {predicted_class}")

print("Unknown Drone Count:", unknown_counter)
print("Known Drone Count:", known_counter)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
Phantom_Test_File1_0_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
Phantom_Test_File2_3_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Phantom_Test_File1_8_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Phantom_Test_File2_13_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Phantom_Test_File2_10_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Phantom_Test_File2_16_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Phantom_Test_File2_24_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Phantom_Test_File2_21_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Phantom_Test_File3_4_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Phantom_Test_File1_0_PS-2_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Phantom_Test_File3_9_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Phantom_Test_File1_8_PS-2_mel.png: Phantom
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Phantom_Test_File2_3_PS-2_mel.png